# Analyse d'algorithmes et programmation – Premier contrôle

## Exercice 1

L'algorithme de tri ci-dessous contient une erreur.

1. Trouver une entrée sur laquelle l'algorithme trie mal,
2. Corriger l'erreur.

In [1]:
def almost_right_sort(liste):
    n = len(liste)
    for i in range(1,n):
        minpos = i
        for j in range(i,n):
            if liste[j] < liste[minpos]:
                minpos = j
        liste[i], liste[minpos] = liste[minpos], liste[i]
    return liste

In [2]:
almost_right_sort([100,401,315,597,126])

[100, 126, 315, 401, 597]

L'algorithme ne trie pas correctement le premier élément de la liste. Le fix est simple: modifier les bornes de la première boucle `for`

In [3]:
almost_right_sort([1000,401,315,597,126])

[1000, 126, 315, 401, 597]

In [4]:
def right_sort(liste):
    n = len(liste)
    for i in range(n):
        minpos = i
        for j in range(i+1,n):
            if liste[j] < liste[minpos]:
                minpos = j
        liste[i], liste[minpos] = liste[minpos], liste[i]
    return liste

In [5]:
right_sort([1000,401,315,597,126])

[126, 315, 401, 597, 1000]

- Combien de comparaisons effectue l'algorithme, en termes de la longueur $n$ de la liste en entrée, au pire cas?

  *Le nombre de comparaisons dépend uniquement de $n$, et non pas du contenu de la liste. Il est évident qu'à la première itération on fait $n$ comparaisons, ensuite $n-1$, etc., pour un total de $(n^2+1)/2 ∈ O(n^2)$ comparaisons. Avec une petite modification aux bornes de la deuxième boucle, l'algorithme reste correcte et fait $n$ comparaisons en moins, ce qui ne change pas l'asymptotique.*

- Combien de comparaisons dans le meilleur cas? *Tout est dans la réponse précédente*

- Combien d'écritures dans la variable `liste`, en termes de la longueur $n$? *Deux par itération de la boucle extérieure, donc $2n$ au total*.

## Exercice 2

L'algorithme ci-dessous trouve le minimum dans une liste

In [6]:
def minimum(liste):
    n = len(liste)
    if n <= 1:
        return liste[0]
    else:
        a = minimum(liste[:n//2])
        b = minimum(liste[n//2:])
        if a < b:
            return a
        else:
            return b

In [7]:
minimum([123, -123, 34, -100, 24, 9, -113, 34, -200, -2, 20])

-200

- Quelle est la complexité de l'algorithme, en termes de la longueur $n$ de la liste en entrée?

  *Notons $M(n)$ la complexité de l'algorithme, alors $M(n) ≈ 2M(n/2) + c$ pour une constante $c$, et $M(1)=d$ pour une autre constante $d$. En déroulant la récurrence, on obtient $M(n) = c + 2c + 4c + \cdots + nd$, et on peut borner la somme géométrique par $2^{\lceil\log_2 n\rceil c}$, donc $M(n) ∈ O(n)$.*

- Modifier l'algorithme pour qu'il retourne les *deux* plus petits éléments de la liste en entrée. Écrire le code ci-dessous.

In [8]:
def minimum2(liste):
    n = len(liste)
    if n == 2:
        return sorted(liste)
    elif n <= 1:
        return liste[0], liste[0]
    else:
        a1, a2 = minimum2(liste[:n//2])
        b1, b2 = minimum2(liste[n//2:])
        return sorted((a1,a2,b1,b2))[:2]

In [9]:
import random
minimum2([random.randint(-1000,1000) for i in range(40)])

[-927, -914]

## Exercice 3

On souhaite stocker quelques centaines d'entiers dans l'intervale $[-1000,1000]$ dans une table de hachage.

Afin de minimiser les risques de collisions, on crée une table d'hachage d'environ $100^2$ cases, par la classe ci-dessous. Pour la fonction de hachage `H`, on veut utiliser une simple réduction modulo un nombre premier proche de $100^2$, disons $9901$; cependant, la fonction $x \pmod{9901}$ serait trop simple, car nous avons fait l'hypothèse que $x∈[-1000,1000]$, et du coup `H` ne prendrait des valeurs que dans un petit sous-ensemble de $\mathbb{Z}/9901\mathbb{Z}$. Afin de bien "melanger" nos entrées, on choisit donc comme fonction de hachage $H(x) = x^{300} + 2\pmod{9901}$.

In [10]:
size = 9901

# on représente le tableau de hachage par une liste de listes
table = [[] for _ in range(size)]

# La fonction de hachage
def H(x):
    return (pow(x, 300, size) + 2) % size
    
def insert(x):
    table[H(x) % size].append(x)

def search(x):
    return x in table[H(x) % size]

# Une fonction pour montrer les cases non-vides de la table
def show():
    for i in range(size):
        if len(table[i]) > 0:
            print(i, table[i])

Voici le résultat après $100$ insertions, chaque case du tableau de hachage est représentée par une liste, la plus part des listes étant vide: `[]`.

- Le nombre moyen de collisions vous parait-il normal ? Comment justifiez-vous ce phenomène ?

  *Par le paradoxe des anniversaires, puisque $100≈\sqrt{9901}$, on s'attend à voir apparaître une collision avec probabilité $1/2$. À vue de nez, il y a beaucoup plus de collisions que ce que la thèorie prévoit.*
  
  *Il est facile de voir que la fonction $H(x)=x^{300}+2$ ne prend pas toutes les valeurs dans $\mathbb{Z}/9901\mathbb{Z}$, en effet $\gcd(300,9900) = 300$, l'image de $x^{300}$ dans $(\mathbb{Z}/9901\mathbb{Z})^\times$ est donc un sous-groupe d'indice $300$, et $x^{300}+2$ est un coset additif de ce sous-groupe.*
  
  *Il suffira de changer la fonction $H$ en une autre fonction polynomiale qui melange mieux. Si on remplace $300$ par un exposant premier à $9900$, on obtiendra une permutation de $\mathbb{Z}/9901\mathbb{Z}$. C'est ce qui est fait plus bas.* 

In [11]:
import random
for i in range(100):
    insert(random.randint(-1000, 1000))
show()

3 [648, 876, 343, -485]
101 [279, -49, -230]
226 [124, -124, -628]
673 [689, -539, 732]
1791 [-363, 743, 666]
2209 [-643, -11, -11]
2300 [266]
2376 [-865, -83, -83, 830, -357]
2400 [-972]
2500 [-669, 338, 375]
2891 [579]
3573 [-262, -374]
4698 [-609, 289, 803, 609]
5098 [434, -764]
5254 [371]
5650 [103, -187, 399, -956]
6996 [482, -921]
7025 [825, 43]
7305 [-649, -903, 649, -226]
7727 [591, 767, -591, 326, -199, 664, 264]
7826 [284, 935, -478, 536, -214]
8132 [-385, 385, 608, -608]
8785 [-726, 708, -542, -143]
8796 [304, -697, -329, -320, -320]
9221 [-867, 545, 524]
9239 [-852, -588, -514, 583, -642]
9460 [917, 812, -792, -87, 721]
9680 [91, 975, -438, 50]
9681 [518, 19]
9682 [479, 213, -479]
9803 [598, 750]


- Définissez une fonction `max_len` donnant la longueur maximale d'une case de la table;
- Définissez une fonction `stats` comptant, pour chaque $n$ entre $0$ et `max_len`, combien de cases contiennent $n$ éléments.
- Modifiez la fonction `H` afin d'avoir une meilleur repartition dans la table de hachage

In [12]:
def max_len():
    return max(len(l) for l in table)

In [13]:
max_len()

7

In [14]:
def stats():
    bins = {}
    for c in table:
        l = len(c)
        bins[l] = bins.get(l, 0) + 1
    return bins

In [15]:
stats()

{0: 9870, 4: 7, 3: 8, 1: 4, 5: 5, 2: 6, 7: 1}

In [16]:
def H(x):
    return (pow(x, 299, size) + x**2 + 2) % size

table = [[] for _ in range(size)]
for i in range(100):
    insert(random.randint(-1000, 1000))
show()

2 [-432]
365 [-124]
549 [-791]
916 [-24]
928 [459]
943 [377]
1028 [-577]
1077 [498]
1164 [-599]
1431 [-162]
1493 [949]
1520 [824]
1679 [889]
1762 [937]
1801 [203]
1962 [-453]
2368 [-930, -930]
2379 [12]
2426 [199]
2501 [394]
2717 [846]
2943 [-495]
2994 [-340]
3401 [225]
3445 [767]
3472 [-188]
3476 [538]
3594 [475, 475]
3669 [603]
3693 [-36]
3731 [828]
4078 [556]
4116 [638]
4392 [-48]
4529 [-568]
4572 [-630]
4625 [983]
4635 [609, 609]
4693 [35]
4703 [-279]
4760 [-752]
4761 [117]
4772 [435]
4907 [-469]
5050 [-212]
5125 [-890]
5320 [699]
5397 [-871]
5488 [-918]
5550 [-316, -316]
5576 [-754]
5623 [662]
5646 [-533]
5775 [-74]
5846 [83]
5926 [-329]
5960 [446]
6079 [-360]
6238 [573]
6288 [-245]
6346 [876]
6361 [-443]
6475 [63]
6486 [-471]
6648 [-762]
6726 [-750]
6831 [47]
6942 [785]
7071 [-998, 332]
7082 [-93]
7173 [809]
7339 [-769]
7345 [-435]
7509 [-287]
7523 [-37]
7632 [-949]
7808 [-796]
7821 [398]
7914 [-855]
7963 [637]
7999 [627]
8021 [604]
8113 [-984]
8256 [-760]
8287 [-606]
8781 [108]


In [17]:
max_len()

2

In [18]:
stats()

{0: 9806, 1: 90, 2: 5}

## Problème

Nous voulons écrire un algorithme pour écrire toutes les expressions algèbriques équivalentes par commutativité à une expression donnée. Par exemple, en ayant comme entrée $$(a + b) · c,$$ nous voulons donner les sorties $$(b + a) · c,$$ $$c · (a + b),$$ $$c · (b + a).$$

Pour cela nous allons nous servir de piles at d'arbres. On commence par définir trois classes, représentant les trois types de nœuds d'un arbre.

In [19]:
class Mul:
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
    # cette fonction "magique" permet de faire un affichage lisible par un humain de la classe
    def __repr__(self):
        return '(%s * %s)' % (self.left, self.right)

class Add:
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
    def __repr__(self):
        return '(%s + %s)' % (self.left, self.right)

class Var:
    def __init__(self, letter):
        self.letter = letter
        
    def __repr__(self):
        return str(self.letter)

Avec ces classes, nous pouvos représenter toute expression algébrique, par exemple:

In [20]:
Mul(Var(3), Mul(Add(Var('a'), Var('b')), Var('c')))

(3 * ((a + b) * c))

- Utiliser ces classes pour écrire les expressions suivantes: $$(a + b) + c,$$ $$a + (b + c),$$ $$(a + b) · (c + d).$$

In [21]:
Add(Add(Var('a'), Var('b')), Var('c'))

((a + b) + c)

In [22]:
Add(Var('a'), Add(Var('b'), Var('c')))

(a + (b + c))

In [23]:
Mul(Add(Var('a'), Var('b')), Add(Var('c'), Var('d')))

((a + b) * (c + d))

### 1. Lecture

Compléter la fonction ci-dessous (remplacer tous les `pass` par du code), qui prend en entrée une chaîne de caractères, et qui construit l'arbre correspondant à l'aide des classes `Mul`, `Add`, `Var`. Si la chaîne ne représente pas une expression arithmétique, la fonction doit donner une erreur. Tester la fonction sur les entrées proposées plus bas, ainsi que sur d'autres entrées proposées par vous.

**Suggestion :** Vous pouvez utiliser un algorithme à pile similaire à celui vu en TD : à chaque fois qu'une parenthèse fermante est rencontrée, on dépile jusqu'à la parenthèse ouvrante correspondante, on construit le nœud de type `Mul` ou `Add` correspondant, et on remet le résultat dans la pile. Si l'expression est bien parenthésée, à la fin de l'algorithme la pile ne contiendra que l'arbre complet.

In [24]:
def parse(expr):
    stack = []
    for c in expr:
        if c in '+*':
            stack.append(c)
        elif c in ' (':
            continue
        elif c == ')':
            r = stack.pop()
            op = stack.pop()
            l = stack.pop()
            if op not in '+*':
                raise RuntimeError('Unknown operation %s' % op)
            if op == '+':
                Noeud = Add
            else:
                Noeud = Mul
            stack.append(Noeud(l, r))
        else:
            stack.append(Var(c))
    assert len(stack) == 1, 'Expression mal formée'
    return stack[0]

In [25]:
parse('(a+b)')

(a + b)

In [26]:
parse('((a+b)*(c+d))')

((a + b) * (c + d))

In [27]:
parse('(a+b')

AssertionError: Expression mal formée

### 2. Permutation

- Écrire un programme qui prend en entrée un arbre formé par les classes `Mul`, `Add`, `Var`, et qui renvoie une copie de l'arbre avec tous les `left` échangés avec `right`.

**Note:** Vous pouvez tester si un objet appartient à une classe donnée avec la fonction `isinstance`, comme cela:

In [28]:
a = Var('a')
isinstance(a, Var)

True

In [29]:
def renverse(arbre):
    if isinstance(arbre, Var):
        return Var(arbre.letter)
    else:
        return arbre.__class__(renverse(arbre.right), renverse(arbre.left))

In [30]:
renverse(parse('((a+b)*(c+d))'))

((d + c) * (b + a))

- Écrire un programme qui prend en entrée un arbre formé par les classes `Mul`, `Add`, `Var`, et qui renvoie **toutes** ses permutations possibles (échanges de `left` et `right`).

In [31]:
def permutations(arbre):
    if isinstance(arbre, Var):
        return [Var(arbre.letter)]
    else:
        lefts = permutations(arbre.left)
        rights = permutations(arbre.right)
        return ([arbre.__class__(l, r) for l in lefts for r in rights]
                + [arbre.__class__(r, l) for l in lefts for r in rights])

In [32]:
permutations(parse('((a+b)*(c+d))'))

[((a + b) * (c + d)),
 ((a + b) * (d + c)),
 ((b + a) * (c + d)),
 ((b + a) * (d + c)),
 ((c + d) * (a + b)),
 ((d + c) * (a + b)),
 ((c + d) * (b + a)),
 ((d + c) * (b + a))]

### 3. Mettre le tout ensemble

- En faisant appel aux fonctions écrites précédemment, écrire un programme qui prend en entrée une expression algébrique et qui en renvoie toutes les expressions équivalentes par commutativité.
- Décrire la complexité de la méthode. Est-elle optimale?

  *Il est naturel de donner la complexité en termes de la longueur $n$ de l'expression en entrée, cependant une mesure plus précise peut être donnée en termes du nombre $b$ d'opérations binaires dans l'expression, et il est clair que $b∈\Theta(n)$.*
  
  *La fonction `parse` ne fait qu'une passe linéaire sur la chaîne de caractères, et pour chaque itération de la boucle elle effectue $O(1)$ opérations, elle a donc complexité $O(n)=O(b)$.*
  
  *La fonction `permutations` donne en sortie $x$ chaînes de caractères de longueur $n$, sa complexité ne peut donc pas être moindre que $\Omega(xn)$. Pour déterminer $x$, on remarque qu'à chaque appel récursif la taille de la sortie double, et que le nombre d'appel récursifs est égal au nombre $b$ d'opérations binaires, on a donc une borne inférieure de $\Omega(2^b n) = \Omega(2^b b)$.*
  
  *On vérifie que la complexité est exactement $\Theta(2^b b)$: notons $P(B)$ le coût de la fonction `permutation` sur un arbre contenant $B$ operations binaires. Si l'arbre est réduit à une feuille de type `Var`, alors $B=0$ et $P(0)=\Theta(1)$; sinon $P(B) = 2P(B-1) +c(2^{B-1})^2(B-1)$ pour une constante $c$. En déroulant la récurrence on obtient la borne voulue.*

In [33]:
def commute(expr):
    return permutations(parse(expr))

In [34]:
commute('((a+b)*(c+d))')

[((a + b) * (c + d)),
 ((a + b) * (d + c)),
 ((b + a) * (c + d)),
 ((b + a) * (d + c)),
 ((c + d) * (a + b)),
 ((d + c) * (a + b)),
 ((c + d) * (b + a)),
 ((d + c) * (b + a))]